# Tutorial: Enzyme Reaction

In [2]:
from IPython.display import HTML

HTML("""
    <center>
    <video src="../../../_static/" width="50%" height="50%" controls>
    </video></center>
""")

In this tutorial, we will perform QM/MM simulations modelling th DNA cleavage mechanism in the HNH-like domain of I-Ppo1. To accelerate the simulation, we utilize the multiple time step approach, (Outer time step: B3LYP/6-31G*, and inner step: PM3), and SINR thermostat. 

Additionally, we will use QMHub to add long-range electrostatics.

We define the reaction coordinate as the difference in bond breaking (P - O) and bond forming9O - P) distances, and calculate the free energy profile using umbrella sampling and MBAR.

## Input Files

Make a new working directory.

```bash
mkdir ippoi
cd ippoi
```

Make an `input` directory. This is where we will store the PDB files and generate Amber inputs.

```bash
mkdir input
cd input
```

Now we can use tleap to make our `.parm7` and `.rst7` files.

```bash
```

Run tleap, and you should get step3_pbcsetup.parm7 and step3_pbcsetup.rst7.

```bash
tleap -sf tleap.in
```

### Restraint Files

A `cv.rst` file will contain atomic restraints for the mechanism we are interested in.

```
```

`# r1 - r2 (CC - CO bond)` Comment describing the restraint. In this case, we are doing restraining the difference between the first distance (r1) and the second distance (r2). 

`iat=3,4,1,6,` Atom numbers for the restraint. The first distance (r1) is the length between atoms 3 and 4. The second distance (r2) is the length between atoms 1 and 6.

`rstwt=1.,-1.,` Indicates we want to take the difference between r1 and r2.

`r1=-99, r2=__REST__, r3=__REST__, r4=99,` The shape of our biasing potential. `__REST__` is a placeholder which we will modify later.

`rk2=150.0, rk3=150.0,` Force constant in kcal/mol.

### QM/MM MD Input files

2 MD input files are used. The first file (step5.00_equilibration.mdin) is used for the initial pathway sampling at a few picoseconds. The second file (step6.00_equilibration.mdin) is used for additional sampling (+10 ps).

1. step5.00_equilibration.mdin
2. step6.00_equilibration.mdin

The general format of a QM/MM MD input file differs from classical MD with the `&qmmm` section.

```bash
A NVT simulation for common production-level simulations
 &cntrl
    imin=0,        ! No minimization
    irest=1,       ! This IS a restart of an old MD simulation
    ntx=5,         ! So our inpcrd file has velocities

    ! Boundary conditions
    ntb=0,         ! Periodic boundaries for constant volume
    ntp=0,         ! No pressure control

    ! Temperature control
    ntt=3,         ! Langevin Mid
    gamma_ln=5.0,  ! Friction coefficient (ps^-1)
    temp0=300.0,   ! Target temperature
    ig=-1,         ! Random number seed

    ! Potential energy control
    cut=999.0,      ! nonbonded cutoff, in Angstroms

    ! MD settings
    nstlim=500,   ! 1 ps total
    dt=0.001,      ! time step (ps)

    ! SHAKE
    ntc=1,         ! Constrain bonds containing hydrogen
    ntf=1,         ! Do not calculate forces of bonds containing hydrogen
    tol=0.000001,  ! Shake tolerance

    ! Control how often information is printed
    ntpr=100,      ! Print energies every 100 steps
    ntwx=100,      ! Print coordinates every 100 steps to the trajectory
    ntwr=100,     ! Print a restart file every 5K steps (can be less frequent)
!   ntwv=-1,       ! Uncomment to also print velocities to trajectory
!   ntwf=-1,       ! Uncomment to also print forces to trajectory
    ntxo=2,        ! Write NetCDF format
    ioutfm=1,      ! Write NetCDF format (always do this!)

    ! Restraints
    nmropt=1,      ! Turn on restraints

    ! QM/MM
    ifqnt=1,       ! Turn on QM/MM
 /


 &qmmm
    ! QM atoms
    qmmask=':1'    ! Amber residue mask for QM atoms

    ! QM settings
    qm_theory='PM3', ! Semiempirical method
    qmcharge=0,    ! Charge of QM subsystem

    ! Shake
    qmshake=0,     ! Use Shake for QM atoms

    ! Potential energy control
    qmcut=999.0,    ! Cutoff for QM/MM electrostatic interactions
    writepdb=1,    ! Check QM atoms
 /

 &wt type='DUMPFREQ', istep1=10 /
 &wt type='END' /
 DISANG=cv.rst
 DUMPAVE=step5.00_equilibration.cv
```

With this we have all of the necessary files. We can go back to the working directory.

```bash
cd ../
```

## Generate Umbrella Windows

The script `gen_inputs.in` will prepare folders for umbrella sampling, and modify the `cv.rst` file so that each window has a different restraint value.

```bash
#!/bin/bash

mkdir -p 00
cd 00/
ln -sf ../input/step3_pbcsetup.parm7
ln -sf ../input/step6.00_equilibration.mdin .
sed -e 's/irest=1/irest=0/;s/ntx=5/nts=1,/' ../input/step5.00_equilibration.mdin > step5.00_equilibration.mdin
cd ..

for i in `seq -w 1 20`; do
mkdir -p $i
cd $i
ln -sf ../input/step3_pbcsetup.parm7
ln -sf ../input/step5.00_equilibration.mdin .
ln -sf ../input/step6.00_equilibration .
cd ..
done

cp ../input/step3_pbcsetup.rst7 00/step5.00_equilibration_inp.rst7

n=-2.0
for i in `seq -f"%02g" 0 20`
do
    nn=$(printf "%.2f" "$n")
    sed "s/__REST__/${nn}/g" input/cv.rst > ${i}/cv.rst
    n=`echo $n + 0.20 | bc`
done
```

## Running QM/MM Simulation

Run QM/MM simulations of the inital pathway, `runqmmm1.slurm`.

```bash
#!/bin/bash
#SBATCH --partition=debug
#SBATCH --exclusive
#SBATCH --ntasks=20
#SBATCH --mem=0
#SBATCH --output=%j.out
#SBATCH --error=%j.err
#SBATCH --time=00:30:00
#SBATCH --job-name=CA

date
module load intel/2020a
source /home/panxl/amber20/amber.sh
export UCX_TLS=all

#SANDER="mpiexec.hydra -bootstrap slurm -n 1 sander.MPI"
SANDER="sander"
init="step3_pbcsetup"

# First Round

istep="step5.00_equilibration"

for i in `seq 0 20`; do
    printf -v j "%02d" $i
    cd $j
    if [ ${i} -eq 0 ]; then
    $SANDER -O -i ${istep}.mdin -o ${istep}.mdout -p ${init}.parm7 -c ${istep}_inp.rst7 -r ${istep}.ncrst -x ${istep}.nc -inf ${istep}.mdinfo
    else
    $SANDER -O -i ${istep}.mdin -o ${istep}.mdout -p ${init}.parm7 -c ${istep}_inp.ncrst -r ${istep}.ncrst -x ${istep}.nc -inf ${istep}.mdinfo
    fi
    printf -v j "%02d" $(($i + 1))
    cp ${istep}.ncrst ../${j}/${istep}_inp.ncrst
    cd ..
done
```

Once this is done, perform additional sampling in each window, `runqmmm2.slurm`.

```bash
#!/bin/bash
#SBATCH --partition=debug
#SBATCH --exclusive
#SBATCH --ntasks=20
#SBATCH --mem=0
#SBATCH --output=%j.out
#SBATCH --error=%j.err
#SBATCH --time=00:30:00
#SBATCH --job-name=CA

date
module load intel/2020a
source /home/panxl/amber20/amber.sh
export UCX_TLS=all

SANDER="sander"
init="step3_pbcsetup"

# First Round

pstep="step5.00_equilibration"
istep="step6.00_equilibration"

for i in `seq -w 0 20`; do
    cd $i
    $SANDER -O -i ${istep}.mdin -o ${istep}.mdout -p ${init}.parm7 -c ${pstep}.ncrst -r ${istep}.ncrst -x ${istep}.nc -inf ${istep}.mdinfo
    cd ..
done
```